# Environment Setup

In [1]:
import sys

import open3d as o3d

# Python version
assert sys.version_info >= (3, 8)
# Open3D version
assert o3d.__version__ >= "0.17.0"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Point Cloud Filtering

## Down-sampling

In [2]:
from tutorials.open3d import down_sample

filename = "data/input/bunny.ply"
down_sample.down_sample(filename)

In [3]:
from tutorials.open3d import visualization

down_sampled_filename = "data/output/bunny - downsampled.ply"
visualization.visualization(down_sampled_filename)